In [1]:
import numpy as np
import pandas as pd
import itertools
from collections import Counter
import json
import matplotlib.pyplot as plt
import imdb
from price_parser import Price
%matplotlib inline

#### Step 1: Preprocessing, data scraping

In [2]:
DATA_COLUMNS = ["index","wikipedia_id", "imdb_id", "title", "cast", "genres", "runtimes", "countries", "languages", "box_office", "rating", "votes", "plot", "synopsis"]

df = pd.read_csv("./Data/fetched_data.csv", names=DATA_COLUMNS, header=0, index_col=0)

In [3]:
from ast import literal_eval

df["title"] = df["title"].astype("string")
df.cast = df.cast.apply(lambda x: literal_eval(x))
df.genres = df.genres.apply(lambda x: [] if pd.isna(x) else literal_eval(x))
df["runtimes"] = pd.to_numeric(df["runtimes"].apply(lambda x: "0" if pd.isna(x) else x.replace("[", "").replace("]", "").replace("'", "")))
df.countries = df.countries.apply(lambda x: [] if pd.isna(x) else literal_eval(x))
df.languages = df.languages.apply(lambda x: [] if pd.isna(x) else literal_eval(x))

In [4]:
pd.to_numeric(df["box_office"][213].split(" ")[1].replace("$", "").replace(",", "").replace("'", ""))

293000

In [5]:
# TODO question assistant => comment avoir la bonne valeur de box office en dollar 
# sachant que le cours a evolue au cours du temps

def convert_currency(x):
    formated = Price.fromstring(x)
    amount = formated.amount_float
    print(formated.currency, amount)
    match formated.currency:
        case "$":
            return amount
        case "EUR":
            return amount * 1.18
        case "GBP":
            return amount * 1.36
        case "SEK":
            return amount * 0.11
        case "CAD":
            return amount * 0.79
        case "INR":
            return amount * 0.013
        case "CZK":
            return amount * 0.04
        case "JPY":
            return amount * 0.009
        case _:
            return None
    

In [6]:
df["budget"] = df["box_office"].apply(lambda x: None if pd.isna(x) else convert_currency(x.split(" ")[1].replace("'", "")))

$ 28000000.0
$ 2000000.0
$ 9000000.0
$ 6000000.0
$ 16000000.0
$ 4000000.0
$ 5000000.0
$ 5000000.0
SEK 40000000.0
$ 8500000.0
$ 9000000.0
$ 200000.0
CAD 185000.0
$ 782000.0
$ 1662000.0
$ 300000.0
$ 26500.0
$ 23500000.0
$ 4100000.0
INR 530000000.0
$ 7000000.0
$ 4000000.0
$ 450000.0
$ 6000000.0
$ 7000000.0
$ 12000000.0
$ 1250000.0
$ 35000.0
$ 13000000.0
$ 3251794.0
$ 800000.0
$ 25000.0
$ 6000000.0
None None
$ 6000000.0
$ 10000000.0
$ 916000.0
$ 1103000.0
$ 2443322.0
$ 16500000.0
$ 4000000.0
$ 15000000.0
$ 2700000.0
$ 22000000.0
$ 55000000.0
$ 1500000.0
$ 55000000.0
ZAR 2400000.0
$ 150000.0
None None
$ 500.0
$ 13500000.0
$ 50000000.0
NLG 1100000.0
$ 31000000.0
$ 25000000.0
INR 40000000.0
$ 125000.0
$ 2600000.0
None None
$ 6000000.0
$ 33000000.0
$ 24500000.0
$ 12800000.0
$ 24000000.0
$ 7000000.0
BDT 3000000.0
None None
None None
None None
$ 5000000.0
$ 807000.0
$ 222000.0
GBP 150000.0
$ 2311000.0
$ 6000000.0
$ 293000.0
$ 2500000.0
None None
$ 500000.0
$ 65000000.0
$ 15000000.0
EUR 4500000.0

In [7]:
df.sample(5)

,wikipedia_id,imdb_id,title,cast,genres,runtimes,countries,languages,box_office,rating,votes,plot,synopsis,budget
index,,,,,,,,,,,,,,
31223,27177006,1194417,casino jack,"[Kevin Spacey, Ruth Marshall, Graham Greene, H...","[Biography, Comedy, Crime, Drama]",108,[Canada],[English],"{'Budget': '$15,000,000 (estimated)'}",6.2,17351,['A hot shot Washington DC lobbyist and his pr...,NaN,15000000.0
19243,26215143,356121,thooval kottaram,"[Jayaram, Manju Warrier, Sukanya, Murali, Dile...","[Comedy, Drama, Romance]",0,[India],[Malayalam],NaN,6.6,203,['When Mohanachandran helps a depressed Devapr...,NaN,NaN
14090,27875557,79135,fascination,"[Franca Maï, Brigitte Lahaie, Jean-Marie Lemai...",[Horror],80,[France],[French],NaN,6.0,3104,['A runaway criminal breaks into an eerie chat...,NaN,NaN
9162,27081302,41093,adventure in baltimore,"[Robert Young, Shirley Temple, John Agar, Albe...","[Comedy, Drama]",89,[United States],[English],NaN,6.1,473,['The liberated daughter of a 1905 minister in...,NaN,NaN
7039,28744996,43409,circle of danger,"[Ray Milland, Patricia Roc, Marius Goring, Hug...","[Drama, Film-Noir, Mystery, Romance]",86,[United Kingdom],[English],NaN,6.3,690,"[""In the 1950s, an American comes to Britain t...",NaN,NaN


In [23]:
# df.genres = df.genres.apply(lambda x: [] if pd.isna(x) else literal_eval(x))
df["synopsis"] = df["synopsis"].apply(lambda x: "" if pd.isna(x) else literal_eval(x)[0])[0:50]

In [25]:
df["synopsis"][0:50]

index
0     200 years in the future, Mars has been coloniz...
1                                                      
2                                                      
3                                                      
4                                                      
5                                                      
6     The film begins with Chorus, in this case a pe...
7     Spoiled and bored upper crust Edwardian Englis...
8                                                      
9                                                      
10                                                     
11                                                     
12                                                     
13                                                     
14                                                     
15                                                     
16                                                     
17    Andrea Wilcox (Zoe Tamerlis) is an a

In [8]:

df["rating"] = pd.to_numeric(df["rating"].apply(lambda x: "0" if pd.isna(x) else x.replace("[", "").replace("]", "").replace("'", "")))

AttributeError: 'float' object has no attribute 'replace'

#### Step 2: Investigating the evolution of movie production over time and finding main character

In [ ]:
# 2.1 movie production

In [ ]:
# main character


In [ ]:
synopsys[0]

'["200 years in the future, Mars has been colonized by a high-tech company.\\nMelanie Ballard (Natasha Henstridge) arrives by train to a Mars mining camp which has cut all communication links with the company headquarters. She\'s not alone, as she is with a group of fellow police officers. They find the mining camp deserted except for a person in the prison, Desolation Williams (Ice Cube), who seems to laugh about them because they are all going to die. They were supposed to take Desolation to headquarters, but decide to explore first to find out what happened.\\n\\nThey find a man inside an encapsulated mining car, who tells them not to open it. However, they do and he tries to kill them. One of the cops witnesses strange men with deep scarred and heavily tattooed faces killing the remaining survivors. The cops realise they need to leave the place fast.\\n\\nDesolation explains that the miners opened a kind of Martian construction in the soil which unleashed red dust. Those who breath

#### Step 3: Box office feature analysis

#### Step 4: Finding correlation

#### Step 5: Predicting future production and box office

#### Step 6: Final visualisation of the datastory